In [1]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [97]:
# get the TPM values
import pandas as pd
tpm_df = pd.read_csv("GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_median_tpm.gct", sep='\t')
gene_name_to_tpm_dict = dict()
for index, row in tpm_df.iterrows():
    gene_name_to_tpm_dict[row["Description"]] = row["Skin - Sun Exposed (Lower leg)"]

In [119]:
# Read in the micro-array data - so far only founds matches between 1230 genes in whitfield in the 56K genes from GTEX
# TODO - get better name mapping
import math
import numpy as np
count = 0
count_found = 0
whitfield_micro_array_df = pd.read_csv("Whitfield_2013_cell_cycle_2830_all_scores.csv")
    
def get_coarse_phase(angle):
    # angles version 2
    predicted_phase = -1
    if angle >= 0 and angle <= 0.8:
        predicted_phase = "G1" #G1_angles += 1
    elif angle > 0.8 and angle <= 2.5:
        predicted_phase = "S" #S_angles += 1
    elif angle > 2.5 and angle <= 5.4:
        predicted_phase = "G2M" #G2M_angles += 1
    elif angle > 5.4:
        predicted_phase = "G1" #G1_angles += 1
    # angles version 1
    #if angle >= 0 and angle <= 2:
    #    predicted_phase = "S"
    #elif angle > 2 and angle <= 4.8:
    #    predicted_phase = "G2M"
    #elif angle > 4.8 and angle <= 5.9:
    #    predicted_phase = "G1"
    #elif angle > 5.9:
    #    predicted_phase = "S"
    return predicted_phase



gene_names_with_micro_array_data = list()

time_course_1_angles_list = list()
time_course_2_angles_list = list()
time_course_3_angles_list = list()
time_course_4_angles_list = list()
time_course_all_phases_list = list()
time_course_all_angles_list = list()
names_for_all_time_courses_list = list()
gene_name_to_time_course_expressions_dict = dict()

# calculate the angles for the 4 time courses
for i in range(0,40,2): #t-course 1 T = 17.65, φ = 0.0
    #print(math.atan2(math.sin((i/17.65)*2*math.pi),math.cos((i/17.65)*2*math.pi))) # with negative radians
    angle = ((i/17.65)*2*math.pi)%(2*math.pi)
    time_course_1_angles_list.append(angle) # 0 to 2pi
    time_course_all_angles_list.append(angle) # 0 to 2pi
    time_course_all_phases_list.append(get_coarse_phase(angle)) # 0 to 2pi
    names_for_all_time_courses_list.append("t1_" + str(i))
for i in range(0,48,2): #t-course 2 T = 18.6, φ = 0.0
    #print(math.atan2(math.sin((i/18.6)*2*math.pi),math.cos((i/18.6)*2*math.pi))) # with negative radians
    angle = ((i/18.6)*2*math.pi)%(2*math.pi)
    time_course_2_angles_list.append(angle) # 0 to 2pi
    time_course_all_angles_list.append(angle) # 0 to 2pi
    time_course_all_phases_list.append(get_coarse_phase(angle))
    names_for_all_time_courses_list.append("t2_" + str(i))
for i in range(0,48,2): #t-course 3 T = 18, φ = 0.0
    #print(math.atan2(math.sin((i/18)*2*math.pi),math.cos((i/18)*2*math.pi))) # with negative radians
    angle = ((i/18)*2*math.pi)%(2*math.pi)
    time_course_3_angles_list.append(angle) # 0 to 2pi
    time_course_all_angles_list.append(angle) # 0 to 2pi
    time_course_all_phases_list.append(get_coarse_phase(angle))
    names_for_all_time_courses_list.append("t3_" + str(i))
for i in range(0,46,2): #t-course 4 T = 23.95, φ = 2.3
    #print(math.atan2(math.sin((i/23.95)*2*math.pi + 2.3),math.cos((i/23.95)*2*math.pi + 2.3))) # with negative radians
    angle = ((i/23.95)*2*math.pi + 2.3)%(2*math.pi)
    time_course_4_angles_list.append(angle) # 0 to 2pi
    time_course_all_angles_list.append(angle) # 0 to 2pi
    time_course_all_phases_list.append(get_coarse_phase(angle))
    names_for_all_time_courses_list.append("t4_" + str(i))    
    
    
for index, row in whitfield_micro_array_df.iterrows():
    count +=1
    name = str(row['NAME'])
    if (name != 'nan'):
        #print("___" + name + "___")
        names = name.split("^")
        if (len(names) > 1):
            #print("___" + names[1] + "___")
            gene_name = names[1].strip()
            if (gene_name in gene_name_to_tpm_dict and not(gene_name in gene_names_with_micro_array_data)):
                
                
                ####!!!! cut-off to have just a few top cycling genes
                scaled_fourier_score = float(row['Scaled Fourier'])
                #if (scaled_fourier_score < 10.0):
                 #   continue
                    
                tpm = gene_name_to_tpm_dict[gene_name]
                #print(gene_name + " is found")
                gene_names_with_micro_array_data.append(gene_name)
                count_found += 1
                gene_name_to_time_course_expressions_dict[gene_name] = dict()
                gene_name_to_time_course_expressions_dict[gene_name]["all"] = list()
                gene_name_to_time_course_expressions_dict[gene_name]["one"] = list()
                
                
                new_row = row[11:31].replace(r'\s+', np.nan, regex=True)
                new_row = new_row.fillna(0)
                row[11:31] = new_row
            
                new_row = row[32:56].replace(r'\s+', np.nan, regex=True)
                new_row = new_row.fillna(0)
                row[32:56] = new_row
                
                new_row = row[57:81].replace(r'\s+', np.nan, regex=True)
                new_row = new_row.fillna(0)
                row[57:81] = new_row
                
                new_row = row[82:105].replace(r'\s+', np.nan, regex=True)
                new_row = new_row.fillna(0)
                row[82:105] = new_row
                
                
                #TODO: for now adding zero to missing time courses and samples - valid for all things?
                
                for i in range(11,31):
                    gene_name_to_time_course_expressions_dict[gene_name]["one"].append((2**row[i]) * tpm if row[i] else 0) # undo the log2 of data
                    gene_name_to_time_course_expressions_dict[gene_name]["all"].append((2**row[i]) * tpm if row[i] else 0) # undo the log2 of data
                gene_name_to_time_course_expressions_dict[gene_name]["two"] = list()
                for i in range(32,56):
                    gene_name_to_time_course_expressions_dict[gene_name]["all"].append((2**row[i]) * tpm if row[i] else 0)
                    gene_name_to_time_course_expressions_dict[gene_name]["two"].append((2**row[i]) * tpm if row[i] else 0) # undo the log2 of data
                gene_name_to_time_course_expressions_dict[gene_name]["three"] = list()
                for i in range(57,81):
                    gene_name_to_time_course_expressions_dict[gene_name]["all"].append((2**row[i]) * tpm if row[i] else 0)
                    gene_name_to_time_course_expressions_dict[gene_name]["three"].append((2**row[i]) * tpm if row[i] else 0) # undo the log2 of data
                gene_name_to_time_course_expressions_dict[gene_name]["four"] = list()
                for i in range(82,105):
                    gene_name_to_time_course_expressions_dict[gene_name]["all"].append((2**row[i]) * tpm if row[i] else 0)
                    gene_name_to_time_course_expressions_dict[gene_name]["four"].append((2**row[i]) * tpm if row[i] else 0) # undo the log2 of data
    
                gene_name_to_time_course_expressions_dict[gene_name]["sine"] = row['Sin']
                gene_name_to_time_course_expressions_dict[gene_name]["cosine"] = row['Cos']
                gene_name_to_time_course_expressions_dict[gene_name]["phase"] = row['Phase'].strip()
                gene_name_to_time_course_expressions_dict[gene_name]["fourier_score"] = scaled_fourier_score
print(count_found)

1230


In [142]:
# data_mESCbulk.h5f 1:1 RUN this cell
import h5py
import pandas as pd

SC_df = pd.DataFrame()
ftst = h5py.File("data_mESCbulk.h5f",'r')
print([key for key in ftst.keys()])
print(ftst['labels'][:])
gene_names_all = ftst['sym_names'][:]
gene_names_all = [x.decode('utf-8').upper() for x in gene_names_all]
print(len(gene_names_all))
YT = ftst['counts'][:]
YT = YT[[0,1,3],:]
print(YT.shape)
labels = ["bulk_G1", "bulk_S", "bulk_G2M"]

SC_df = pd.DataFrame(YT.T)
SC_df.columns = labels
SC_df.index = gene_names_all
SC_df = SC_df[~SC_df.index.duplicated(keep='first')]
print(SC_df.shape)

['class_labels', 'counts', 'ensIds', 'labels', 'sym_names']
[1. 2. 4. 3.]
38293
(3, 38293)
(38050, 3)


In [149]:
# MouseEST 35 1:3 RUN this cell for mouseEST
import numpy as np
import h5py
ftst = h5py.File("normCounts_mESCquartz.h5f",'r')
print([key for key in ftst.keys()])
print(ftst['cell_names'][:])
labels_tst = ftst['phase_vecS'][:]*2+ftst['phase_vecG2M'][:]*3+ftst['phase_vecG1'][:]
labels_tst = np.array([int(i) for i in labels_tst])
print(labels_tst)

['KG1', 'KG2M', 'KS', 'K_phase', 'LogNcountsQuartz', 'LogVar_techQuartz_logfit', 'ccCBall_gene_indices', 'ccGO_gene_indices', 'cell_names', 'cellcyclegenesGO', 'gene_names_all', 'genes_heterogen', 'phase_vecG1', 'phase_vecG2M', 'phase_vecS', 'sym_namesQ']
[b'ES_G1_1' b'ES_G1_2' b'ES_G1_3' b'ES_G1_4' b'ES_G1_5' b'ES_G1_6'
 b'ES_G1_7' b'ES_G1_8' b'ES_G1_9' b'ES_G1_10' b'ES_G1_11' b'ES_G1_12'
 b'ES_G1_13' b'ES_G1_14' b'ES_G1_15' b'ES_G1_16' b'ES_G1_17' b'ES_G1_18'
 b'ES_G1_19' b'ES_G1_20' b'ES_S_1' b'ES_S_2' b'ES_S_3' b'ES_S_4' b'ES_S_5'
 b'ES_S_6' b'ES_S_7' b'ES_G2M_1' b'ES_G2M_2' b'ES_G2M_3' b'ES_G2M_4'
 b'ES_G2M_5' b'ES_G2M_6' b'ES_G2M_7' b'ES_G2M_8']
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3]


In [150]:
# MouseEST 35 2:3 RUN this cell for mouseEST
# a bit slow, try not to re-run
import mygene
mg = mygene.MyGeneInfo()

gene_names_all = ftst['gene_names_all'][:]
gene_names_all = [x.decode('utf-8') for x in gene_names_all]
geneSyms = mg.querymany(gene_names_all, scopes='ensembl.gene', fields='symbol', species='mouse')
newGeneList = list()
for i in range(0, len(gene_names_all)):
    if 'symbol' in geneSyms[i]:
        newGeneList.append(geneSyms[i]['symbol'].upper())
    else:
        newGeneList.append(gene_names_all[i])

YT = ftst['LogNcountsQuartz'][:]
print(YT.shape)
labels_tst = ftst['phase_vecS'][:]*2+ftst['phase_vecG2M'][:]*3+ftst['phase_vecG1'][:]
print(labels_tst)
print(ftst["cell_names"][:])
# G1, S, G2M

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-18000...done.
querying 18001-19000...done.
querying 19001-20000...done.
querying 20001-21000...done.
querying 21001-22000...done.
querying 22001-23000...done.
querying 23001-24000...done.
querying 24001-25000...done.
querying 25001-26000...done.
querying 26001-27000...done.
querying 27001-28000...done.
querying 28001-29000...done.
querying 29001-30000...done.
querying 30001-31000...done.
querying 31001-32000...done.
querying 32001-33000...done.
querying 33001-34000...done.
querying 34001-35000...done.
queryin

In [151]:
# MouseEST 35 3:3 RUN this cell for mouseEST
string_labels = list()
cell_names = ftst["cell_names"][:]
cell_names = gene_names_all = [x.decode('utf-8') for x in cell_names]
for label in cell_names:
    if ("G1" in label):
        string_labels.append("G1")
    if ("S" in label):
        string_labels.append("S")
    if ("G2" in label):
        string_labels.append("G2")
SC_df = pd.DataFrame(YT.T)
SC_df.columns = cell_names
SC_df.index = newGeneList

print(SC_df.shape)
SC_df = SC_df[~SC_df.index.duplicated(keep='first')]
print(SC_df.shape)
print(SC_df["ES_G1_1"]["BFAR"])

(36807, 35)
(36793, 35)
0.9468431505995761


In [154]:
# Stahlberg 1:1 RUN this cell for Stahlberg
import pandas as pd

# read in the single-cell data, and identify the set of genes that also have micro-array/tpm expression (1114)
# for each cell, get percentage of reads for each gene
# for the micro-array set, get percentage of "expression" for each gene
# for each gene, get the best match of percentages for each of the time-courses, and take an average angle
# then take the average angle of all the genes - that gives the phase
### process SC single cell data for Seurat input
SC_df = pd.read_csv("Stahlberg_MCF7_group_1_to_6.csv", sep=',')
def phase_to_string(phase_number):
    if (phase_number in (1, 4, 9)):
        return "G1"
    if (phase_number in (2, 5, 10)):
        return "S"
    if (phase_number in (3, 6, 11)):
        return "G2M"
SC_df = SC_df.rename(lambda x: x + "_" + phase_to_string(int(SC_df.loc[x, "#Group"])))
SC_df = SC_df.iloc[:,:-3]
SC_df = SC_df.replace(-1,0.0).T
print(len(SC_df.columns))
print(len(SC_df.index))

90
81


In [146]:
# microarray as SC leg normalized, RUN This 1:1
import pandas as pd
SC_df = pd.read_csv("Skin-SunExposed(Lowerleg)_whitfield_micro_array_as_sc.csv", sep='\t', index_col = 0)

new_list = list()
for col,phase in zip(SC_df.columns, time_course_all_phases_list):
    new_list.append(col+ "_"+phase)
SC_df.columns = new_list
print(SC_df.index)
print(SC_df.columns)
print(gene_names_with_micro_array_data)

Index(['KIF20A', 'HSPA1A', 'PLK1', 'MYB', 'CCNB1', 'CCNE2', 'CDCA8', 'SGOL2',
       'ASPM', 'TROAP',
       ...
       'TM4SF1', 'PER1', 'FAM3C', 'CKS1B', 'GUCY1B3', 'APOBEC3C', 'AMICA1',
       'BCL11B', 'PPP2R5E', 'DDX11'],
      dtype='object', length=1230)
Index(['t1_0_G1', 't1_2_G1', 't1_4_S', 't1_6_S', 't1_8_G2M', 't1_10_G2M',
       't1_12_G2M', 't1_14_G2M', 't1_16_G1', 't1_18_G1', 't1_20_S', 't1_22_S',
       't1_24_S', 't1_26_G2M', 't1_28_G2M', 't1_30_G2M', 't1_32_G2M',
       't1_34_G1', 't1_36_G1', 't1_38_S', 't2_0_G1', 't2_2_G1', 't2_4_S',
       't2_6_S', 't2_8_G2M', 't2_10_G2M', 't2_12_G2M', 't2_14_G2M', 't2_16_G1',
       't2_18_G1', 't2_20_G1', 't2_22_S', 't2_24_S', 't2_26_S', 't2_28_G2M',
       't2_30_G2M', 't2_32_G2M', 't2_34_G2M', 't2_36_G1', 't2_38_G1',
       't2_40_S', 't2_42_S', 't2_44_S', 't2_46_G2M', 't3_0_G1', 't3_2_G1',
       't3_4_S', 't3_6_S', 't3_8_G2M', 't3_10_G2M', 't3_12_G2M', 't3_14_G2M',
       't3_16_G1', 't3_18_G1', 't3_20_G1', 't3_22_S', 't3_24_

In [139]:
# 930 Dataset, RUN This 1:1

import csv

SC_df = pd.DataFrame()

column_names = []
with open('Dataset930.csv') as f:
    reader = csv.reader(f, delimiter="\t")
    next(reader)
    counter = 0
    full_row = []
    gene_name = ""
    for row in reader:
        if (counter < 930):
            phase = ""
            if (row[1] == "g0/g1"):
                phase = "G1"
            elif (row[1] == "s"):
                phase = "S"
            elif (row[1] == "g2/m"):
                phase = "G2M"
            name = "cell_" + str(counter) + "_" + phase
            column_names.append(name)
        if (counter%930 == 0 and counter > 0):
            SC_df[gene_name] = full_row
            #print(full_row)
            full_row = []
            
        full_row.append(2**float(row[9]))
        gene_name = row[6]
        
        counter += 1
SC_df = SC_df.T
SC_df.columns = column_names
print(SC_df.index)
print(SC_df.shape)

Index(['ABCF1', 'ALAS1', 'ANLN', 'ARL6IP1', 'ATF2', 'BCL6', 'BIRC5', 'BMP1',
       'BTBD3', 'BUB1',
       ...
       'TUBB', 'UBE2C', 'UNG', 'USP1', 'VANGL1', 'VCAM1', 'VPS25', 'WDR76',
       'ZC3HC1', 'ZNF367'],
      dtype='object', length=252)
(252, 930)


In [155]:
# this is version 5 v 5 v5 attempt at linear regression
import statistics
import datetime
now = datetime.datetime.now()
print(str(now.hour) + " " + str(now.minute) + " " + str(now.second))
import sklearn
from sklearn import linear_model
SC_our_assignment_tupples_list = list()
for cell_name in SC_df:
    
    # get a set of genes that are non-zero in that cell
    count = 0
    whitfield_subset_micro_array_df = pd.DataFrame(columns = range(0,91))
    cell_cycle_genes_for_analysis = list()
    cell_data = []
    for gene_name, row in SC_df.iterrows():
        if ((gene_name in gene_names_with_micro_array_data) and SC_df[cell_name][gene_name]):
            count += 1
            cell_cycle_genes_for_analysis.append(gene_name)
            cell_data.append(SC_df[cell_name][gene_name])
            whitfield_subset_micro_array_df.loc[gene_name] = gene_name_to_time_course_expressions_dict[gene_name]["all"]
    print(str(count) + " genes used for cell " + cell_name)
    #print(cell_cycle_genes_for_analysis)
    cell_data_array = np.array(cell_data)
    #print(cell_data_array[:5])
    # huge, and pretty uniform MSE for all time points when fitting individual cells
    mse_sum = 0
    best_mse = math.inf
    best_angle = -1
    for i in range(0, 91): 
        lm = linear_model.LinearRegression()
        #print(whitfield_subset_micro_array_df.iloc[:, [i]])
        lm.fit(whitfield_subset_micro_array_df.iloc[:, [i]], cell_data_array)
        #print(cell_data_array)
        #print(lm.predict(whitfield_subset_micro_array_df.iloc[:, [i]]))
        mse = np.mean((cell_data_array - lm.predict(whitfield_subset_micro_array_df.iloc[:, [i]])) **2)
        #print("MSE of using just a column fit for the first column:")
        #print(mse)
        mse_sum += mse
        if (mse < best_mse):
            best_mse = mse
            best_angle = time_course_all_angles_list[i]
    
    # using the entire micro-array for a single fit results in a higher average MSE, but test it out later
    # to see if by chance the accuracy is higher... - did better for the bulk, at least!!!!
    #print("MSE of using a jsut a column for each column, Average")
    #print(mse_sum/91)
    lm = linear_model.LinearRegression()
    lm.fit(whitfield_subset_micro_array_df, cell_data_array)
    #print(lm.intercept_)
    #print(lm.coef_)
    #print(whitfield_subset_micro_array_df.iloc[:, [0]])
    mse_sum = 0
    best_mse = math.inf
    best_angle = -1
    for i in range(0, 91): 
    #    print(i)
    #    print(whitfield_subset_micro_array_df.iloc[:, [i]][:5])
        new = whitfield_subset_micro_array_df.iloc[:, [i]] * lm.coef_[i] + lm.intercept_
    #    #print(new)
        mse = np.mean((cell_data_array - np.array(new)) **2)
        mse_sum += mse
        if (mse < best_mse):
            best_mse = mse
            best_angle = time_course_all_angles_list[i]
    #print("MSE of using a whole array fit, Average")
    #print(mse_sum/91)
    
    angle = best_angle
    predicted_phase = -1
    # Angles - Version 2
    # angles version 2
    if angle >= 0 and angle <= 0.8:
        predicted_phase = "G1" #G1_angles += 1
    elif angle > 0.8 and angle <= 2.5:
        predicted_phase = "S" #S_angles += 1
    elif angle > 2.5 and angle <= 5.4:
        predicted_phase = "G2M" #G2M_angles += 1
    elif angle > 5.4:
        predicted_phase = "G1" #G1_angles += 1
    # Angles - Version 1
    # angles version 2
    #if angle >= 0 and angle <= 2:
    #    predicted_phase = "S"
    #elif angle > 2 and angle <= 4.8:
    #    predicted_phase = "G2M"
    #elif angle > 4.8 and angle <= 5.9:
    #    predicted_phase = "G1"
    #elif angle > 5.9:
    #    predicted_phase = "S"
    
    print(predicted_phase)
    SC_our_assignment_tupples_list.append((cell_name, predicted_phase))

15 27 18
10 genes used for cell A1_G1
G1
12 genes used for cell A2_G1
G1
16 genes used for cell A3_S
S
23 genes used for cell A4_S
G1
22 genes used for cell A5_G2M
S
18 genes used for cell A6_G2M
G2M
19 genes used for cell A7_G1
G2M
18 genes used for cell A8_G1
G2M
19 genes used for cell A9_S
G1
17 genes used for cell A10_S
G1
19 genes used for cell A11_G2M
G2M
19 genes used for cell A12_G2M
G2M
14 genes used for cell A2b_G1
G2M
18 genes used for cell B2_G1
G2M
19 genes used for cell A4b_S
S
23 genes used for cell B5_G2M
S
7 genes used for cell B6_G2M
G2M
12 genes used for cell A8b_G1
G1
9 genes used for cell B8_G1
G1
18 genes used for cell B9_S
S
18 genes used for cell B10_S
G2M
19 genes used for cell B11_G2M
G1
9 genes used for cell B12_G2M
G1
11 genes used for cell C1_G1
G2M
8 genes used for cell C2_G1
S
16 genes used for cell C3_S
S
20 genes used for cell A12b_G2M
S
10 genes used for cell C1b_G1
G1
21 genes used for cell C6_G2M
S
16 genes used for cell C7_G1
G2M
10 genes used for c

In [156]:
S_tp = 0
S_fp = 0
S_fn = 0
G2M_tp = 0
G2M_fp = 0
G2M_fn = 0
G1_tp = 0
G1_fp = 0
G1_fn = 0
num_s = 0
num_g2 = 0
num_g1 = 0
 
#SC_our_assignment_tupples_list = [("1_G1", "G2M"), ("2_S", "G1"), ("3_G2M","G2M")]
for (cell_name, predicted_phase) in SC_our_assignment_tupples_list:
    if ("_S" in cell_name):
        num_s += 1
        if (predicted_phase == "S"):
            S_tp += 1
        else:
            S_fn += 1
            if (predicted_phase == "G2M"):
                G2M_fp += 1
            elif (predicted_phase == "G1"):
                G1_fp += 1
    elif ("_G2" in cell_name):   
        num_g2 += 1
        if (predicted_phase == "G2M"):
            G2M_tp += 1        
        else:
            G2M_fn += 1
            if (predicted_phase == "S"):
                S_fp += 1
            elif (predicted_phase == "G1"):
                G1_fp += 1
    elif ("_G1" in cell_name):
        num_g1 += 1
        if (predicted_phase == "G1"):
            G1_tp += 1
        else:
            G1_fn += 1
            if (predicted_phase == "S"):
                S_fp += 1
            elif (predicted_phase == "G2M"):
                G2M_fp += 1 

S_precision = S_tp / (S_tp + S_fp) if S_tp else 0
S_recall = S_tp / (S_tp + S_fn) if S_tp else 0
G2M_precision = G2M_tp / (G2M_tp + G2M_fp) if G2M_tp else 0
G2M_recall = G2M_tp / (G2M_tp + G2M_fn) if G2M_tp else 0
G1_precision = G1_tp / (G1_tp + G1_fp) if G1_tp else 0
G1_recall = G1_tp / (G1_tp + G1_fn) if G1_tp else 0
precision_avg = (S_precision + G2M_precision + G1_precision)/3
recall_avg = (S_recall + G2M_recall + G1_recall)/3
f1_score = 2*((precision_avg*recall_avg)/(precision_avg+recall_avg))
print("F1_score")
print(f1_score)
print("num S: " + str(num_s))
print("num G2: " + str(num_g2))
print("num G1: " + str(num_g1))

F1_score
0.3880184067088499
num S: 29
num G2: 30
num G1: 31


In [33]:
test_df = pd.DataFrame()
test_df["gene1"] = [100,200]
test_df["gene2"] = [38, 12]
print(test_df.loc[0])
test_cell = [50, 2]
import sklearn
from sklearn import linear_model
lm = linear_model.LinearRegression()
print(test_cell)

gene1    100
gene2     38
Name: 0, dtype: int64
[50, 2]


In [50]:
lm.fit(test_df.loc[0].reshape(-1, 1), test_cell)
print(lm.intercept_)
print(lm.coef_)
mse = np.mean((test_cell - lm.predict(test_df.loc[0].reshape(-1, 1))) **2)
print(mse)

-27.419354838709666
[0.77419355]
2.524354896707238e-29


/home/moamin/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  if __name__ == '__main__':
/home/moamin/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
